## BMI for the bionic arm

As a proof of concept we want to show that our bionic arm can replicate the actions performed by the monkey during a trial.

### Import

In [1]:
import paho.mqtt.client as paho
import numpy as np
from time import sleep
from datetime import datetime
import json

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Parameters

In [2]:
clientID = 'bmi'
broker = 'localhost'
port = 1883
topic = 'brain_activity'

### Script

In [3]:
# Initialize the publisher
client = paho.Client(clientID, False)

# Initialize buffer
global buffer
buffer = None

# Initialize model
#model = 
def myOnMessageReceived(paho_mqtt, userdata, msg):
    global buffer
    if msg.topic == topic:
        message = json.loads(msg.payload)
        if message['status'] == 'starting_simulation':
            print('Simulation started!')
            buffer = np.empty((message['shape'],))
            
        elif message['status'] == 'running_simulation':
            buffer = np.vstack((buffer, message['value']['measurements']))
            #phase, obj = model.predict(buffer.T)
            phase=0
            obj=0
            print(message['value']['index'], '# ', buffer.shape, phase, obj)
        
        elif message['status'] == 'ending_simulation':
            print('Simulation terminated')
            buffer = None
            
            
        
client.on_message = myOnMessageReceived

In [ ]:
client.connect(broker, port)
client.loop_start()
client.subscribe(topic, 2)
print('Waiting for simulation...')
while True:
    try:
        sleep(1)
        
    except KeyboardInterrupt:
        print('Simulation interrupted')
        client.unsubscribe(topic)
        client.loop_stop()
        client.disconnect()
        sys.exit(0)

Waiting for simulation...
Simulation started!
0 #  (2, 552) 0 0
1 #  (3, 552) 0 0
2 #  (4, 552) 0 0
3 #  (5, 552) 0 0
4 #  (6, 552) 0 0
5 #  (7, 552) 0 0
6 #  (8, 552) 0 0
7 #  (9, 552) 0 0
8 #  (10, 552) 0 0
9 #  (11, 552) 0 0
10 #  (12, 552) 0 0
11 #  (13, 552) 0 0
12 #  (14, 552) 0 0
13 #  (15, 552) 0 0
14 #  (16, 552) 0 0
Simulation terminated


## 📲 Initialize the XBee radio module

Specify which serial port the transmitting Xbee radio module is connected to:

In [4]:
XBEE_PORT = 'COM5'

> ℹ️ **NOTE:** On Linux, the port name of the XBee radio module (transmitter) will be
   - `/dev/ttyACMX` when using e.g. Arduino + shield;
   - `/dev/ttyUSBX` when using a dedicated USB interface.

Import the XBee modules and create an XBee device object with the serial port we defined earlier and a baud rate of 115200:

In [5]:
from digi.xbee.devices import XBeeDevice, RemoteXBeeDevice, XBee64BitAddress

xbee = XBeeDevice(XBEE_PORT, 115200)

Open the connection to the XBee device:

In [6]:
xbee.open()

Print the node name and MAC address of this XBee device so we know we have the correct one:

In [7]:
print("Node ID: %s \t MAC: %s" % (xbee.get_node_id(), xbee.get_64bit_addr()) )

Node ID: TRANSMITTER 	 MAC: 0013A20041CB9200


Try to find the aggregator node on the network so we can send data to it later:

In [9]:
#xbee_network = xbee.get_network()

#aggregator_xbee = xbee_network.get_device_by_node_id("AGGREGATOR")

AGGREGATOR_MAC = "0013A20041C14D4A"
aggregator_xbee = RemoteXBeeDevice(xbee, XBee64BitAddress.from_hex_string(AGGREGATOR_MAC))

#if aggregator_xbee is None:
#    raise ValueError('Aggregator node not found!')

In [10]:
#import struct

payload = bytearray([0, 1, 2, 255])

#payload.extend(bytearray(struct.pack("f", resonance_freq)))
#payload.extend(bytearray(struct.pack("f", resonance_ampl))) 


Let us print the payload as hexadecimal values.
The first four values correspond to the resonance frequency and the last four correspond to the resonance amplitude.

In [11]:
print([ "0x%02x" % b for b in payload ])

['0x00', '0x01', '0x02', '0xff']


## 📡 Transmit 802.15.4 packet with XBee radio module

Send the payload that we defined earlier to the aggregator/receiving XBee:

In [12]:
tx_status = xbee.send_data(aggregator_xbee, payload)

print(tx_status)


{<DictKeys.HEADER_BYTE: 'header'>: 126, <DictKeys.LENGTH: 'length'>: 3, <DictKeys.FRAME_SPEC_DATA: 'fr_spec_data'>: {<DictKeys.FRAME_TYPE: 'fr_type'>: <ApiFrameType.TX_STATUS: (137, 'TX (Transmit) Status')>, <DictKeys.FRAME_ID: 'fr_id'>: 9, <DictKeys.API_DATA: 'api_spec_data'>: {<DictKeys.TS_STATUS: 'ts_status'>: <TransmitStatus.SUCCESS: (0, 'Success.')>}}, <DictKeys.CHECKSUM: 'checksum'>: 109}


In [ ]:
# 5 fingers + wrist + elbow + approaching
actuator_dict = {
    "ball":[175,175,175,175,175, 80, 255, X],
}

payload = bytearray(actuator_dict[pred]+)

